# Inicio

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("bank_train.csv")
df_clean = df.copy()
df_test = pd.read_csv("bank_test.csv")
df_test_clean = df_test.copy()

## Mapeamento

In [24]:
#Treino
df_clean['y'] = df_clean['y'].replace({"yes": 1, "no": 0})
df_clean['loan'] = df_clean['loan'].replace({'no': 0, 'n': 0, 'y':1, 'yes':1})
df_clean['default'] = df_clean['default'].replace({'no': 0, 'yes':1})
df_clean['housing'] = df_clean['housing'].replace({'no': 0, 'yes':1})
#Teste
df_test_clean['loan'] = df_test_clean['loan'].replace({'no': 0, 'n': 0, 'y':1, 'yes':1})
df_test_clean['default'] = df_test_clean['default'].replace({'no': 0, 'yes':1})
df_test_clean['housing'] = df_test_clean['housing'].replace({'no': 0, 'yes':1})

#MAPEAMENTOS
#Treino
month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

# Aplicar o mapeamento à coluna 'month'
df_clean['month'] = df_clean['month'].map(month_mapping)

#Teste

month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

# Aplicar o mapeamento à coluna 'month'
df_test_clean['month'] = df_test_clean['month'].map(month_mapping)

education_mapping = {
    'primary': 1, 'secondary': 2, 'tertiary': 3
}

# Aplicar o mapeamento à coluna 'month'
df_clean['education'] = df_clean['education'].map(education_mapping)
#Teste
df_test_clean['education'] = df_test_clean['education'].map(education_mapping)

marital_mapping = {
    'single': 1, 'married': 2, 'divorced': 3
}

df_clean['marital'] = df_clean['marital'].map(marital_mapping)
#Teste
df_test_clean['marital'] = df_test_clean['marital'].map(marital_mapping)

contact_mapping = {
    'cellular': 1, 'unknown': 0, 'telephone': 2
}

df_clean['contact'] = df_clean['contact'].map(contact_mapping)
#Teste
df_test_clean['contact'] = df_test_clean['contact'].map(contact_mapping)

job_mapping = {
    'management':1, 'blue-collar':2, 'student':3, 'self-employed':4, 'technician':5,
 'administrator':6, 'retired':7, 'housemaid':8, 'services':9, 'unemployed':10,
 'admin.':6, 'unknown':0, 'entrepreneur':11 
}

df_clean['job'] = df_clean['job'].map(job_mapping)
#Teste
df_test_clean['job'] = df_test_clean['job'].map(job_mapping)
#aa


C:\Users\jvmil\AppData\Local\Temp\ipykernel_28372\1280514580.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['y'] = df_clean['y'].replace({"yes": 1, "no": 0})
C:\Users\jvmil\AppData\Local\Temp\ipykernel_28372\1280514580.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['loan'] = df_clean['loan'].replace({'no': 0, 'n': 0, 'y':1, 'yes':1})
C:\Users\jvmil\AppData\Local\Temp\ipykernel_28372\1280514580.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. T

## Limpeza de colunas


In [25]:
#Treino
df_clean = df_clean.drop(columns=['location'])
df_clean = df_clean.drop(columns=['poutcome'])
df_clean = df_clean.drop(columns=['id'])
#Teste
df_test_clean = df_test_clean.drop(columns=['location'])
df_test_clean = df_test_clean.drop(columns=['poutcome'])

In [26]:
df_clean = df_clean[df_clean['job'] != 'unknown']
df_clean = df_clean[df_clean['education'] != 'unknown']


## Remoção de outliers


In [27]:
df_clean = df_clean[(df_clean['balance'] < 15000) & (df_clean['balance'] > -5000) ]
df_clean = df_clean[(df_clean['age'] <= 80) & (df_clean['age'] > 18)]
df_clean = df_clean[(df_clean['previous'] <= 30) ]


In [28]:

df_clean = df_clean.dropna(subset=['marital'])
df_clean = df_clean.dropna(subset=['job'])
df_clean = df_clean.dropna(subset=['contact'])
df_clean = df_clean.dropna(subset=['education'])
df_clean = df_clean.dropna(subset=['previous'])
df_clean = df_clean.dropna(subset=['campaign'])
df_clean = df_clean.dropna(subset=['campaign'])


# Funções auxiliares

## Função de output

In [29]:
def predictions(model):
    X_test_final = df_test_clean.drop(columns=['id'])
    ids = df_test_clean['id']

    # Fazer previsões
    y_pred_final = model.predict(X_test_final)

    # Criar um DataFrame com os resultados
    results = pd.DataFrame({
        'id': ids,
        'y': y_pred_final
    })

    # Salvar os resultados em um arquivo CSV
    results.to_csv('predictions.csv', index=False)
    print('Predictions saved to predictions.csv')

## Função de threshhold 


In [30]:
def predictions_threshhold(model,valor):
    X_test_final = df_test_clean.drop(columns=['id'])
    ids = df_test_clean['id']

    # Fazer previsões
    y_scores = model.predict_proba(X_test_final)[:, 1]
    y_pred_adjusted = (y_scores >= valor).astype(int)

    # Criar um DataFrame com os resultados
    results = pd.DataFrame({
        'id': ids,
        'y': y_pred_adjusted
    })

    # Salvar os resultados em um arquivo CSV
    results.to_csv('predictions.csv', index=False)
    print('Predictions saved to predictions.csv')

In [31]:
def predictions_threshhold_alterado_dataSet(model,valor,df_teste):
    X_test_final = df_teste.drop(columns=['id'])
    ids = df_teste['id']

    # Fazer previsões
    y_scores = model.predict_proba(X_test_final)[:, 1]
    y_pred_adjusted = (y_scores >= valor).astype(int)

    # Criar um DataFrame com os resultados
    results = pd.DataFrame({
        'id': ids,
        'y': y_pred_adjusted
    })

    # Salvar os resultados em um arquivo CSV
    results.to_csv('predictions.csv', index=False)
    print('Predictions saved to predictions.csv')

In [32]:
from sklearn.metrics import precision_score, recall_score, f1_score

def findBestTresh(model, X_test):
    y_scores = model.predict_proba(X_test)[:, 1]

    #valores a serem testados
    
    best_f1 = 0
    best_threshold = 0
    best_precision = 0
    best_recall = 0
    
    for c in range(20,50):
        threshold = c / 100
        y_pred_adjusted = (y_scores >= threshold).astype(int)
        precision = precision_score(y_test, y_pred_adjusted, pos_label=1)
        recall = recall_score(y_test, y_pred_adjusted, pos_label=1)
        f1 = f1_score(y_test, y_pred_adjusted, pos_label=1)
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_precision = precision
            best_recall = recall
    
    #print(f"Optimal Threshold: {best_threshold}")
    #print(f"Best Precision: {best_precision:.2f}")
    #print(f"Best Recall: {best_recall:.2f}")
    #print(f"Best F1 Score: {best_f1:.2f}")
    
    return best_threshold

In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score

def findBestTreshByPrecision(model, X_test):
    y_scores = model.predict_proba(X_test)[:, 1]

    #valores a serem testados
    thresholds = [0.2,0.25,0.27, 0.271, 0.272, 0.273, 0.274, 0.275, 0.276, 0.277, 0.278, 0.279,0.3,0.35,0.4
                  ,0.45,0.5]
    
    best_f1 = 0
    best_threshold = 0
    best_precision = 0
    best_recall = 0
    
    for threshold in thresholds:
        y_pred_adjusted = (y_scores >= threshold).astype(int)
        precision = precision_score(y_test, y_pred_adjusted, pos_label=1)
        recall = recall_score(y_test, y_pred_adjusted, pos_label=1)
        f1 = f1_score(y_test, y_pred_adjusted, pos_label=1)
        
        if precision > best_precision:
            best_f1 = f1
            best_threshold = threshold
            best_precision = precision
            best_recall = recall
    
    print(f"Optimal Threshold: {best_threshold}")
    print(f"Best Precision: {best_precision:.2f}")
    print(f"Best Recall: {best_recall:.2f}")
    print(f"Best F1 Score: {best_f1:.2f}")
    
    return best_threshold

# Escolha do modelo (lazypredict)

In [ ]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)


In [ ]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
XGBClassifier                      0.90               0.71     0.71      0.89   
DecisionTreeClassifier             0.87               0.71     0.71      0.88   
LGBMClassifier                     0.90               0.69     0.69      0.89   
BaggingClassifier                  0.89               0.67     0.67      0.88   
RandomForestClassifier             0.90               0.66     0.66      0.89   
PassiveAggressiveClassifier        0.78               0.66     0.66      0.81   
AdaBoostClassifier                 0.89               0.65     0.65      0.88   
NearestCentroid                    0.66               0.64     0.64      0.72   
QuadraticDiscriminantAnalysis      0.79               0.64     0.64      0.81   
ExtraTreeClassifier                0.85               0.63     0.63      0.85   
GaussianNB                  

# Testes com o modelo XGBClassifier

In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

X = df_clean.drop(columns=['y'])
y = df_clean['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)


Normalização dos dados

In [133]:
from sklearn.preprocessing import StandardScaler

cols_to_normalize = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

scaler = StandardScaler()
X_train[cols_to_normalize] = scaler.fit_transform(X_train[cols_to_normalize])
X_test[cols_to_normalize] = scaler.transform(X_test[cols_to_normalize])

KeyError: "['day'] not in index"

Treinando modelo e testando acurácia

In [14]:
from sklearn.metrics import accuracy_score, classification_report
## TREINO DE MODELO

from xgboost import XGBClassifier

modelXGBC = XGBClassifier()
modelXGBC.fit(X_train, y_train)

#TESTANDO ACURACIA

y_pred = modelXGBC.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)
predictions(modelXGBC)


Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.96      0.95      5660
           1       0.58      0.48      0.52       711

    accuracy                           0.90      6371
   macro avg       0.76      0.72      0.73      6371
weighted avg       0.90      0.90      0.90      6371

Predictions saved to predictions.csv


## Alterando hiperparâmetros

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

xgb = XGBClassifier(objective = 'binary:logistic', use_label_encoder = False, eval_metric = 'logloss')

param_grid = {
   'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.02, 0.05, 0.1],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0]
}

# Usar GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros:", grid_search.best_params_)

# Treinar o modelo com os melhores hiperparâmetros
best_params = grid_search.best_params_
best_model = XGBClassifier(**best_params)
best_model.fit(X_train, y_train)

In [105]:
y_pred_best = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_best)
print(f'Accuracy: {accuracy:.2f}')

report = classification_report(y_test, y_pred_best)
print('Classification Report:\n', report)

predictions(best_model)

Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.95      5635
           1       0.61      0.46      0.53       737

    accuracy                           0.90      6372
   macro avg       0.77      0.71      0.74      6372
weighted avg       0.89      0.90      0.90      6372

Predictions saved to predictions.csv


## Teste com oversampling

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

X_train = df_clean.drop(columns=['y'])
y_train = df_clean['y']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

GridSearch para hiperparâmetros

In [ ]:
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier(objective = 'binary:logistic', use_label_encoder = False, eval_metric = 'logloss')

grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.02, 0.05, 0.1],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0]}

grid_search = GridSearchCV(estimator = xgb, param_grid = grid, cv = 5, scoring = 'f1', verbose = 3, n_jobs = -1)
grid_search.fit(X_train_res, y_train_res)

print("Melhores hiperparâmetros:", grid_search.best_params_)

# Melhor modelo
best_params = grid_search.best_params_

In [ ]:
best_model = XGBClassifier(objective = 'binary:logistic', use_label_encoder = False, eval_metric = 'logloss', **best_params)
best_model.fit(X_train_res, y_train_res)

# Prever no conjunto de teste
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia: %.2f%%" % (accuracy * 100.0))

# Relatório de classificação
print(classification_report(y_test, y_pred))

predictions(best_model)

## Teste deletando a coluna de dias

In [15]:
df_clean_sem_dias = df_clean.copy()
df_clean_sem_dias = df_clean_sem_dias.drop(columns=['day'])

Treino de modelo XGBC

In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

X = df_clean_sem_dias.drop(columns=['y'])
y = df_clean_sem_dias['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

Teste de previsão

In [19]:
from sklearn.metrics import accuracy_score, classification_report
## TREINO DE MODELO

from xgboost import XGBClassifier

modelXGBC = XGBClassifier()
modelXGBC.fit(X_train, y_train)

#TESTANDO ACURACIA

y_pred = modelXGBC.predict(X_test)
print("Coluna sem dias")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)

Coluna sem dias
Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.95      5660
           1       0.59      0.45      0.51       711

    accuracy                           0.90      6371
   macro avg       0.76      0.71      0.73      6371
weighted avg       0.89      0.90      0.90      6371



# Teste separando a idade em faixas

### Alterando as idades em faixas

In [11]:
bins = [0, 24,30, 59, 100]  # Definindo os limites das faixas
labels = [1, 2, 3,4]  # Rótulos para cada faixa

# Função para mapear as idades ignorando NaNs
def map_age(age):
    if pd.isna(age):
        return age
    else:
        return pd.cut([age], bins=bins, labels=labels, right=True)[0]
    
df_clean_faixa_idade = df_clean.copy()
df_clean_faixa_idade['age'] = df_clean_faixa_idade['age'].apply(map_age)
df_test_clean_faixa_idade = df_test_clean.copy()



# Aplicar a função de mapeamento
df_test_clean_faixa_idade['age'] = df_test_clean_faixa_idade['age'].apply(map_age)


Criação do modelo

In [10]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

X = df_clean_faixa_idade.drop(columns=['y'])
y = df_clean_faixa_idade['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

NameError: name 'df_clean_faixa_idade' is not defined

### Normalização


In [48]:
df_clean_faixa_idade['balance'] = np.sign(df_clean_faixa_idade['balance']) * np.log1p(np.abs(df_clean_faixa_idade['balance']))

### Oversampling

In [84]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

X_train = df_clean_faixa_idade.drop(columns=['y'])
y_train = df_clean_faixa_idade['y']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)


smote = SMOTE(sampling_strategy=0.55, k_neighbors=3, random_state=22)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [68]:
from sklearn.metrics import accuracy_score, classification_report
## TREINO DE MODELO

from xgboost import XGBClassifier

modelXGBC = XGBClassifier()
modelXGBC.fit(X_train_res, y_train_res)

#TESTANDO ACURACIA

y_pred = modelXGBC.predict(X_test)
print("Coluna faixa idade")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)
predictions(modelXGBC)

Coluna faixa idade
Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.94      5588
           1       0.62      0.49      0.55       782

    accuracy                           0.90      6370
   macro avg       0.77      0.73      0.75      6370
weighted avg       0.89      0.90      0.90      6370

Predictions saved to predictions.csv


### Teste com probabilidade alterada

In [69]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

#treinar modelo
modelXGBC = XGBClassifier()
modelXGBC.fit(X_train_res, y_train_res)
#threshhold
m = findBestTresh(modelXGBC,X_test)
y_scores = modelXGBC.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= m).astype(int)
y_pred = modelXGBC.predict(X_test)
#prints
print("Coluna faixa idade")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)
predictions(modelXGBC)
print("teste")
print(classification_report(y_test, y_pred_adjusted))
predictions_threshhold(modelXGBC,m)


Optimal Threshold: 0.21
Best Precision: 0.51
Best Recall: 0.77
Best F1 Score: 0.61
Coluna faixa idade
Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.94      5588
           1       0.62      0.49      0.55       782

    accuracy                           0.90      6370
   macro avg       0.77      0.73      0.75      6370
weighted avg       0.89      0.90      0.90      6370

Predictions saved to predictions.csv
teste
              precision    recall  f1-score   support

           0       0.97      0.90      0.93      5588
           1       0.51      0.77      0.61       782

    accuracy                           0.88      6370
   macro avg       0.74      0.83      0.77      6370
weighted avg       0.91      0.88      0.89      6370

Predictions saved to predictions.csv


### Otimização com hiperparâmetros

In [9]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# Supondo que X_train_res e y_train_res já estejam definidos

# Definir o classificador e o grid de parâmetros
xgb = XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')

# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [3, 5, 7, 10],
#     'learning_rate': [0.01, 0.02, 0.05, 0.1],
#     'reg_alpha': [0.0, 0.1, 0.5, 1.0],
#     'subsample': [0.6, 0.8, 1.0],
#     'colsample_bytree': [0.6, 0.8, 1.0],
#     'min_child_weight': [1, 3, 5],
#     'gamma': [0, 0.1, 0.3, 0.5]
# }

param_grid = {
    'n_estimators': [100],
    'max_depth': [10],
    'learning_rate': [0.02],
    'reg_alpha': [0.0],
    'subsample': [1.0],
    'colsample_bytree': [1.0],
    'min_child_weight': [1],
    'gamma': [0]
}

# Usar GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)
grid_search.fit(X_train_res, y_train_res)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros:", grid_search.best_params_)

# Treinar o modelo com os melhores hiperparâmetros
best_params = grid_search.best_params_
best_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
best_model.fit(X_train_res, y_train_res)


NameError: name 'X_train_res' is not defined

Hiperparametros rodando 

In [89]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

#treinar modelo
best_model.fit(X_train, y_train)
#threshhold
m = findBestTresh(best_model,X_test)
y_scores = best_model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= m).astype(int)
y_pred = best_model.predict(X_test)
#prints
print("Coluna faixa idade")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)
print("teste")
print(classification_report(y_test, y_pred_adjusted))

predictions_threshhold_alterado_dataSet(best_model,m,df_test_clean_faixa_idade)


C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [12:47:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Coluna faixa idade
Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95      5630
           1       0.65      0.34      0.45       739

    accuracy                           0.90      6369
   macro avg       0.79      0.66      0.70      6369
weighted avg       0.89      0.90      0.89      6369

teste
              precision    recall  f1-score   support

           0       0.97      0.91      0.94      5630
           1       0.53      0.76      0.63       739

    accuracy                           0.90      6369
   macro avg       0.75      0.83      0.78      6369
weighted avg       0.92      0.90      0.90      6369

Predictions saved to predictions.csv


# Alterando loan e housing

## Clonagem de datapath

In [2]:
df_clean_loan_housing = df_clean.copy()
df_clean_loan_housing.loc[df_clean_loan_housing['housing'] == 1, 'loan'] = 1

In [73]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

X = df_clean_loan_housing.drop(columns=['y'])
y = df_clean_loan_housing['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

## Oversampling

In [74]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

X_train = df_clean_loan_housing.drop(columns=['y'])
y_train = df_clean_loan_housing['y']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)


smote = SMOTE(sampling_strategy=0.55, k_neighbors=7, random_state=22)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

## Hiperparâmetros

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

xgb = XGBClassifier(objective = 'binary:logistic', use_label_encoder = False, eval_metric = 'logloss')

# param_grid = {
#    'n_estimators': [100, 200, 300],
#     'max_depth': [3, 5, 7, 10],
#     'learning_rate': [0.01, 0.02, 0.05, 0.1],
#     'reg_alpha': [0.0, 0.1, 0.5, 1.0]
# }
param_grid = {
    'n_estimators': [100],
    'max_depth': [10],
    'learning_rate': [0.02],
    'reg_alpha': [0.0],
    'subsample': [1.0],
    'colsample_bytree': [1.0],
    'min_child_weight': [1],
    'gamma': [0]
}
# Usar GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)
grid_search.fit(X_train_res, y_train_res)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros:", grid_search.best_params_)

# Treinar o modelo com os melhores hiperparâmetros
best_params = grid_search.best_params_
best_model = XGBClassifier(**best_params)
best_model.fit(X_train_res, y_train_res)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [15:02:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Melhores hiperparâmetros: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.02, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 0.0, 'subsample': 1.0}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [75]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
#threshhold
m = findBestTresh(best_model,X_test)
y_scores = best_model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= m).astype(int)
y_pred = best_model.predict(X_test)
#prints
print("Coluna faixa idade")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)
print("teste")
print(classification_report(y_test, y_pred_adjusted))

predictions_threshhold(best_model,m)

Optimal Threshold: 0.29
Best Precision: 0.50
Best Recall: 0.72
Best F1 Score: 0.59
Coluna faixa idade
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.94      5588
           1       0.62      0.37      0.46       782

    accuracy                           0.89      6370
   macro avg       0.77      0.67      0.70      6370
weighted avg       0.88      0.89      0.88      6370

teste
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      5588
           1       0.50      0.72      0.59       782

    accuracy                           0.88      6370
   macro avg       0.73      0.81      0.76      6370
weighted avg       0.90      0.88      0.89      6370

Predictions saved to predictions.csv


# Validação Cruzada


In [88]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

data, target = df_clean_faixa_idade.drop(columns = ['y']), df_clean_faixa_idade['y']

kf = StratifiedKFold(n_splits=5)
final_predictions = []
final_targets = []
best_thresholds = []

for train_index, test_index in kf.split(data, target):
    
    # Dividir os dados em conjuntos de treino e teste
    X_train, X_test = data.iloc[train_index], data.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Aplicação do oversampling no conjunto de treino
    smote = SMOTE(sampling_strategy=0.55, k_neighbors=7, random_state=22)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    # Treinamento do modelo
    model = XGBClassifier()
    model.fit(X_resampled, y_resampled)
    
    m = findBestTresh(model,X_test)
    best_thresholds.append(m)

    y_scores = model.predict_proba(X_test)[:, 1]
    y_pred_adjusted = (y_scores >= m).astype(int)
    
    final_predictions.extend(y_pred_adjusted)
    final_targets.extend(y_test)
    print()

average_threshold = np.mean(best_thresholds)
print(classification_report(final_targets, final_predictions))
predictions_threshhold_alterado_dataSet(model, average_threshold,df_test_clean_faixa_idade)






              precision    recall  f1-score   support

           0       0.97      0.90      0.93     28152
           1       0.50      0.76      0.61      3694

    accuracy                           0.89     31846
   macro avg       0.74      0.83      0.77     31846
weighted avg       0.91      0.89      0.89     31846

Predictions saved to predictions.csv


In [128]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

X = df_clean_faixa_idade.drop(columns=['y'])
y = df_clean_faixa_idade['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

smote = SMOTE(sampling_strategy=0.55, k_neighbors=7, random_state=22)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [129]:
#treinar modelo
model.fit(X_train_res, y_train_res)
#threshhold

y_scores = model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= average_threshold).astype(int)
y_pred = model.predict(X_test)

print("teste")
print(classification_report(y_test, y_pred_adjusted))
predictions_threshhold(model,average_threshold)

teste
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      5588
           1       0.52      0.74      0.61       782

    accuracy                           0.88      6370
   macro avg       0.74      0.82      0.77      6370
weighted avg       0.91      0.88      0.89      6370

Predictions saved to predictions.csv


In [142]:
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import ParameterGrid
from xgboost import XGBClassifier

# Definindo o espaço de hiperparâmetros
param_grid = {
    'n_estimators': [100, 200, 300],           # Menos opções para número de árvores
    'learning_rate': [0.01, 0.05, 0.1],       # Três níveis de taxa de aprendizado
    'max_depth': [3, 5],                      # Dois níveis de profundidade máxima das árvores
    'subsample': [0.8, 1.0],                  # Dois tamanhos de amostra
    'colsample_bytree': [0.8, 1.0],           # Dois níveis de features usadas para cada árvore
    'gamma': [0, 0.1],                        # Dois valores para a regularização gamma
    'min_child_weight': [1, 3]                # Dois níveis de peso mínimo da folha
}

best_f1 = 0
best_params = None
best_model = None

# Loop sobre todas as combinações de hiperparâmetros
for params in ParameterGrid(param_grid):
    model = XGBClassifier(**params)
    model.fit(X_train_res, y_train_res)
    
    m = findBestTresh(model,X_test)
    # Obter as probabilidades previstas para a classe positiva
    y_scores = model.predict_proba(X_test)[:, 1]
    # Aplicar o threshold fixo
    y_pred_adjusted = (y_scores >= average_threshold).astype(int)
    
    # Calcular o F1 Score com o threshold ajustado
    f1 = f1_score(y_test, y_pred_adjusted)
    
    # Armazenar o melhor modelo e hiperparâmetros
    if f1 > best_f1:
        best_f1 = f1
        best_params = params
        best_model = model


y_scores = best_model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= average_threshold).astype(int)

# Resultados finais
print(f"Melhores parâmetros: {best_params}")
print(f"Melhor F1 Score ajustado: {best_f1}")
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_adjusted))

predictions_threshhold(best_model,average_threshold)


KeyboardInterrupt: 

In [91]:
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import f1_score, classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

# Definição dos dados e rótulos
data, target = df_clean_faixa_idade.drop(columns=['y']), df_clean_faixa_idade['y']

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# Aplicação do oversampling no conjunto de treino
smote = SMOTE(sampling_strategy=0.55, k_neighbors=7, random_state=22)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Definição do espaço de hiperparâmetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
}

best_f1 = 0
best_params = None
best_model = None
best_threshold = None

# Loop sobre todas as combinações de hiperparâmetros
for params in ParameterGrid(param_grid):
    model = XGBClassifier(**params)
    model.fit(X_train_res, y_train_res)
    
    # Encontre o melhor threshold para o conjunto de teste
    y_scores = model.predict_proba(X_test)[:, 1]
    best_threshold = findBestTresh(model, X_test)
    y_pred_adjusted = (y_scores >= best_threshold).astype(int)

    f1 = f1_score(y_test, y_pred_adjusted)
 
    # Armazenar o melhor modelo, hiperparâmetros e threshold
    if f1 > best_f1:
        best_f1 = f1
        best_params = params
        best_model = model
        best_threshold = best_threshold

# Relatório de classificação final com o melhor modelo e threshold
y_scores = best_model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= best_threshold).astype(int)

print(f"Melhores parâmetros: {best_params}")
print(f"Melhor F1 Score ajustado: {best_f1}")
print(f"Melhor Threshold: {best_threshold}")
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_adjusted))


Melhores parâmetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 1.0}
Melhor F1 Score ajustado: 0.6085918854415274
Melhor Threshold: 0.32
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      5638
           1       0.54      0.70      0.61       732

    accuracy                           0.90      6370
   macro avg       0.75      0.81      0.77      6370
weighted avg       0.91      0.90      0.90      6370



# Teste maluco


In [92]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

data, target = df_clean_faixa_idade.drop(columns = ['y']), df_clean_faixa_idade['y']

kf = StratifiedKFold(n_splits=5)
final_predictions = []
final_targets = []
best_thresholds = []
parametros = []
best_f1 = 0
best_params = None
best_model = None
best_tresh = None

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
    'reg_alpha': [0.01],       # Regularização L1
    'reg_lambda': [1.0]     # Regularização L2
}

for train_index, test_index in kf.split(data, target):
    
    # Dividir os dados em conjuntos de treino e teste
    X_train, X_test = data.iloc[train_index], data.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Aplicação do oversampling no conjunto de treino
    smote = SMOTE(sampling_strategy=0.55, k_neighbors=7, random_state=22)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    for params in ParameterGrid(param_grid):
        model = XGBClassifier(**params)
        model.fit(X_train_res, y_train_res)
        m = findBestTresh(model,X_test)
    
        y_scores = model.predict_proba(X_test)[:, 1]
        y_pred_adjusted = (y_scores >= m).astype(int)

        f1 = f1_score(y_test, y_pred_adjusted)
     
        if f1 > best_f1:
            best_f1 = f1
            best_params = params
            best_model = model
            best_tresh = m
 
    best_thresholds.append(best_tresh)
    parametros.append(best_params)

    y_scores = best_model.predict_proba(X_test)[:, 1]
    y_pred_adjusted = (y_scores >= best_tresh).astype(int)
    
    final_predictions.extend(y_pred_adjusted)
    final_targets.extend(y_test)
    print(classification_report(y_test, y_pred_adjusted))
    print(best_params)
    print(best_tresh)
    print()

average_threshold = np.mean(best_thresholds)
best_model = XGBClassifier(**best_params)
best_model.fit(data, target)

y_scores = best_model.predict_proba(data)[:, 1]
y_pred_final = (y_scores >= average_threshold).astype(int)

print("Modelo final")
print(classification_report(target, y_pred_final))
predictions_threshhold_alterado_dataSet(best_model,average_threshold,df_test_clean_faixa_idade)



              precision    recall  f1-score   support

           0       0.97      0.94      0.96      5631
           1       0.65      0.81      0.72       739

    accuracy                           0.93      6370
   macro avg       0.81      0.87      0.84      6370
weighted avg       0.94      0.93      0.93      6370

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'reg_alpha': 0.01, 'reg_lambda': 1.0, 'subsample': 0.8}
0.35

              precision    recall  f1-score   support

           0       0.97      0.95      0.96      5631
           1       0.68      0.78      0.72       738

    accuracy                           0.93      6369
   macro avg       0.82      0.87      0.84      6369
weighted avg       0.94      0.93      0.93      6369

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'reg_alpha': 0.01, 'reg_lambda': 1.0, 'subsample': 0.8}
0.35

              precision    recall  f1-score   support

           0       0.97      0.94      0.96    

In [170]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

X = df_clean_faixa_idade.drop(columns=['y'])
y = df_clean_faixa_idade['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

smote = SMOTE(sampling_strategy=0.55, k_neighbors=7, random_state=22)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [172]:
#treinar modelo
best_params_avg = {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}

final_model = XGBClassifier(**best_params_avg)
final_model.fit(X_train_res, y_train_res)
#threshhold

y_scores = final_model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= average_threshold).astype(int)

print("teste")
print(classification_report(y_test, y_pred_adjusted))
predictions_threshhold(final_model,average_threshold)

teste
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      5588
           1       0.58      0.65      0.61       782

    accuracy                           0.90      6370
   macro avg       0.77      0.79      0.78      6370
weighted avg       0.90      0.90      0.90      6370

Predictions saved to predictions.csv


# Validação cruzada com busca de hiperparâmetros

In [108]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import make_scorer, f1_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

data, target = df_clean_faixa_idade.drop(columns = ['y']), df_clean_faixa_idade['y']

kf = StratifiedKFold(n_splits=5)
final_predictions = []
final_targets = []
best_thresholds = []
best_params = []

param_grid = {
    'n_estimators': [200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0]
}

for train_index, test_index in kf.split(data, target):
    
    # Dividir os dados em conjuntos de treino e teste
    X_train, X_test = data.iloc[train_index], data.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Aplicação do oversampling no conjunto de treino
    smote = SMOTE(sampling_strategy=0.55, k_neighbors=7, random_state=22)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    # Treinamento do modelo
    model = XGBClassifier()
    f1_scorer = make_scorer(f1_score, pos_label=1)
    grid_search = GridSearchCV(model, param_grid, scoring=f1_scorer, cv=3)
    grid_search.fit(X_resampled, y_resampled)
    
    best_model = grid_search.best_estimator_
    best_params.append(grid_search.best_params_)
    
    y_scores = best_model.predict_proba(X_test)[:, 1]
    m = findBestTresh(best_model,X_test)
    best_thresholds.append(m)
    y_pred_adjusted = (y_scores >= m).astype(int)
    
    final_predictions.extend(y_pred_adjusted)
    final_targets.extend(y_test)
    print()

average_threshold = np.mean(best_thresholds)
best_params_avg = {k: np.mean([param[k] for param in best_params]) for k in best_params[0]}

print(best_params_avg)

print(classification_report(final_targets, final_predictions))

Optimal Threshold: 0.42
Best Precision: 0.48
Best Recall: 0.70
Best F1 Score: 0.57

Optimal Threshold: 0.32
Best Precision: 0.43
Best Recall: 0.78
Best F1 Score: 0.55

Optimal Threshold: 0.42
Best Precision: 0.47
Best Recall: 0.68
Best F1 Score: 0.55

Optimal Threshold: 0.36
Best Precision: 0.45
Best Recall: 0.71
Best F1 Score: 0.55

Optimal Threshold: 0.4
Best Precision: 0.48
Best Recall: 0.73
Best F1 Score: 0.58

{'learning_rate': 0.01, 'max_depth': 7.0, 'n_estimators': 200.0, 'reg_alpha': 0.02}
              precision    recall  f1-score   support

           0       0.96      0.89      0.92     28152
           1       0.46      0.72      0.56      3694

    accuracy                           0.87     31846
   macro avg       0.71      0.80      0.74     31846
weighted avg       0.90      0.87      0.88     31846



In [109]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

X = df_clean_faixa_idade.drop(columns=['y'])
y = df_clean_faixa_idade['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

smote = SMOTE(sampling_strategy=0.55, k_neighbors=3, random_state=22)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [110]:
#treinar modelo
best_params_avg = {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 200, 'reg_alpha': 0.02}

final_model = XGBClassifier(**best_params_avg)
final_model.fit(X_train_res, y_train_res)
#threshhold

y_scores = final_model.predict_proba(X_test)[:, 1]
y_pred_adjusted = (y_scores >= average_threshold).astype(int)
y_pred = final_model.predict(X_test)

print("teste")
print(classification_report(y_test, y_pred_adjusted))
predictions_threshhold(final_model,average_threshold)

teste
              precision    recall  f1-score   support

           0       0.96      0.88      0.92      5588
           1       0.46      0.71      0.56       782

    accuracy                           0.86      6370
   macro avg       0.71      0.80      0.74      6370
weighted avg       0.90      0.86      0.88      6370

Predictions saved to predictions.csv


Validação cruzada com GPT

In [40]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import numpy as np

# Supondo que df_clean_faixa_idade já esteja carregado

data, target = df_clean_faixa_idade.drop(columns=['y']), df_clean_faixa_idade['y']
kf = StratifiedKFold(n_splits=9, shuffle=True, random_state=22)

# Inicializar variáveis para armazenar os resultados
accuracies = []
precisions = []
recalls = []
f1s = []

# Função findBestTresh (supondo que ela já existe)

for train_index, test_index in kf.split(data, target):
    # Dividir os dados em conjuntos de treino e teste
    X_train, X_test = data.iloc[train_index], data.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Aplicação do oversampling no conjunto de treino
    smote = SMOTE(sampling_strategy=0.55, k_neighbors=7, random_state=22)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    # Treinamento do modelo
    best_model.fit(X_resampled, y_resampled)
    m = findBestTresh(best_model, X_test)
    y_scores = best_model.predict_proba(X_test)[:, 1]
    y_pred_adjusted = (y_scores >= m).astype(int)
    
    # Avaliar o modelo
    accuracies.append(accuracy_score(y_test, y_pred_adjusted))
    precisions.append(precision_score(y_test, y_pred_adjusted))
    recalls.append(recall_score(y_test, y_pred_adjusted))
    f1s.append(f1_score(y_test, y_pred_adjusted))
    print(classification_report(y_test, y_pred_adjusted))
# Mostrar os resultados agregados
# print(f'Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}')
# print(f'Precision: {np.mean(precisions):.4f} ± {np.std(precisions):.4f}')
# print(f'Recall: {np.mean(recalls):.4f} ± {np.std(recalls):.4f}')
# print(f'F1 Score: {np.mean(f1s):.4f} ± {np.std(f1s):.4f}')

# Imprimir o relatório de classificação final
print(classification_report(y_test, y_pred_adjusted))
predictions_threshhold(best_model,m)


Optimal Threshold: 0.28
Best Precision: 0.48
Best Recall: 0.81
Best F1 Score: 0.60
              precision    recall  f1-score   support

           0       0.97      0.88      0.93      3128
           1       0.48      0.81      0.60       411

    accuracy                           0.87      3539
   macro avg       0.72      0.85      0.76      3539
weighted avg       0.91      0.87      0.89      3539

Optimal Threshold: 0.32
Best Precision: 0.45
Best Recall: 0.73
Best F1 Score: 0.56
              precision    recall  f1-score   support

           0       0.96      0.89      0.92      3128
           1       0.45      0.73      0.56       411

    accuracy                           0.87      3539
   macro avg       0.71      0.81      0.74      3539
weighted avg       0.90      0.87      0.88      3539

Optimal Threshold: 0.36
Best Precision: 0.48
Best Recall: 0.71
Best F1 Score: 0.57
              precision    recall  f1-score   support

           0       0.96      0.90      0.9

# Excluir linhas do dataset para ver se altera

## Função de alteração

In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE

# Função para avaliar a remoção de colunas
def teste_excluir_colunas(df, target_column,model,X_train_res,y_train_res,X_test):
    results = {}
    
    # Separar as features e o target
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    y_pred = model.predict(X_test)
    base_score = f1_score(y_test, y_pred)
    
    results['base_model'] = base_score
    
    # Avaliar a remoção de cada coluna
    for column in X.columns:
        X_temp = X.drop(columns=[column])
        
        X_train, X_test, y_train, y_test = train_test_split(X_temp, y, test_size=0.2, random_state=22)
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
        
        model.fit(X_train_res, y_train_res)
        y_pred = model.predict(X_test)
        score = f1_score(y_test, y_pred)
        
        results[column] = score
    
    return results

In [49]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
df_clean_sem_coluna = df_clean.copy()
X = df_clean_sem_coluna.drop(columns=['y'])
y = df_clean_sem_coluna['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

Oversampling

In [46]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy=0.55, k_neighbors=3, random_state=22)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

Hiperparametros

In [37]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

# Supondo que X_train_res e y_train_res já estejam definidos

# Definir o classificador e o grid de parâmetros
xgb = XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')

# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [3, 5, 7, 10],
#     'learning_rate': [0.01, 0.02, 0.05, 0.1],
#     'reg_alpha': [0.0, 0.1, 0.5, 1.0],
#     'subsample': [0.6, 0.8, 1.0],
#     'colsample_bytree': [0.6, 0.8, 1.0],
#     'min_child_weight': [1, 3, 5],
#     'gamma': [0, 0.1, 0.3, 0.5]
# }

param_grid = {
    'n_estimators': [100],
    'max_depth': [10],
    'learning_rate': [0.02],
    'reg_alpha': [0.0],
    'subsample': [1.0],
    'colsample_bytree': [1.0],
    'min_child_weight': [1],
    'gamma': [0]
}

# Usar GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)
grid_search.fit(X_train_res, y_train_res)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros:", grid_search.best_params_)

# Treinar o modelo com os melhores hiperparâmetros
best_params = grid_search.best_params_
best_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
best_model.fit(X_train_res, y_train_res)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [09:02:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Melhores hiperparâmetros: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.02, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 0.0, 'subsample': 1.0}


C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [09:02:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.02, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

Função de threshhold + gerar resultado

In [47]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

def resultado_threshold(modelo_treinado):
    #threshhold
    m = findBestTresh(modelo_treinado,X_test)
    y_scores = modelo_treinado.predict_proba(X_test)[:, 1]
    y_pred_adjusted = (y_scores >= m).astype(int)
    y_pred = modelo_treinado.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy:.2f}')
    report = classification_report(y_test, y_pred)
    print('Classification Report:\n', report)
    print(classification_report(y_test, y_pred_adjusted))

## Código final

In [51]:
resultado = teste_excluir_colunas(df_clean_sem_coluna,'y',best_model,X_train_res,y_train_res,X_test)
print(resultado)

ValueError: feature_names mismatch: ['age', 'job', 'marital', 'education', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous'] ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous']
training data did not have the following fields: default

gepeto

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, classification_report, precision_recall_curve
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

# Função para encontrar o melhor limiar de decisão baseado na métrica F1

# Função para avaliar a remoção de colunas
def teste_excluir_colunas(df, target_column, model):
    results = {}
    
    # Separar as features e o target
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Treinar e avaliar o modelo com todas as colunas
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)
    smote = SMOTE(sampling_strategy=0.55, k_neighbors=3, random_state=22)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    model.fit(X_train_res, y_train_res)
    m = findBestTresh(model, X_test)
    y_scores = model.predict_proba(X_test)[:, 1]
    y_pred_adjusted = (y_scores >= m).astype(int)
    base_score = f1_score(y_test, y_pred_adjusted)
    
    results['base_model'] = base_score
    
    # Avaliar a remoção de cada coluna
    for column in X.columns:
        X_temp = X.drop(columns=[column])
        
        X_train, X_test, y_train, y_test = train_test_split(X_temp, y, test_size=0.2, random_state=22)
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
        
        model.fit(X_train_res, y_train_res)
        y_scores = model.predict_proba(X_test)[:, 1]
        y_pred_adjusted = (y_scores >= m).astype(int)
        score = f1_score(y_test, y_pred_adjusted)
        print("Coluna não usada: ",column)
        resultado_threshold(best_model, X_test, y_test,m)
        print("Resultado sem thredhhold alterado")
        resultado(best_model, X_test, y_test,m)
        
        results[column] = score
    
    return results
# Preparar os dados
df_clean_sem_coluna = df_clean.copy()
X = df_clean_sem_coluna.drop(columns=['y'])
y = df_clean_sem_coluna['y']

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

# Aplicar oversampling
smote = SMOTE(sampling_strategy=0.55, k_neighbors=3, random_state=22)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Função para imprimir o relatório de classificação e acurácia
def resultado_threshold(modelo_treinado, X_test, y_test,m):
    y_scores = modelo_treinado.predict_proba(X_test)[:, 1]
    y_pred_adjusted = (y_scores >= m).astype(int)
    accuracy = accuracy_score(y_test, y_pred_adjusted)
    print(f'Accuracy: {accuracy:.2f}')
    report = classification_report(y_test, y_pred_adjusted)
    print('Classification Report:\n', report)

def resultado(modelo_treinado, X_test, y_test,m):
    y_scores = modelo_treinado.predict_proba(X_test)[:, 1]
    y_pred_adjusted = (y_scores >= 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred_adjusted)
    print(f'Accuracy: {accuracy:.2f}')
    report = classification_report(y_test, y_pred_adjusted)
    print('Classification Report:\n', report)

# Avaliar a remoção de colunas
resultado = teste_excluir_colunas(df_clean_sem_coluna, 'y', best_model)
print(resultado)



C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [09:51:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [09:51:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Coluna não usada:  age
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93      5588
           1       0.53      0.66      0.59       782

    accuracy                           0.89      6370
   macro avg       0.74      0.79      0.76      6370
weighted avg       0.90      0.89      0.89      6370

Resultado sem thredhhold alterado
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      5588
           1       0.57      0.54      0.56       782

    accuracy                           0.89      6370
   macro avg       0.75      0.74      0.75      6370
weighted avg       0.89      0.89      0.89      6370



C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [09:51:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Coluna não usada:  job
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93      5588
           1       0.53      0.67      0.59       782

    accuracy                           0.89      6370
   macro avg       0.74      0.79      0.76      6370
weighted avg       0.90      0.89      0.89      6370

Resultado sem thredhhold alterado
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      5588
           1       0.58      0.54      0.56       782

    accuracy                           0.89      6370
   macro avg       0.76      0.74      0.75      6370
weighted avg       0.89      0.89      0.89      6370



C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [09:51:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


...

Coluna não usada:  campaign
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93      5588
           1       0.53      0.67      0.59       782

    accuracy                           0.89      6370
   macro avg       0.74      0.79      0.76      6370
weighted avg       0.90      0.89      0.89      6370

Resultado sem thredhhold alterado
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      5588
           1       0.57      0.55      0.56       782

    accuracy                           0.89      6370
   macro avg       0.76      0.75      0.75      6370
weighted avg       0.89      0.89      0.89      6370



C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [09:51:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Coluna não usada:  pdays
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93      5588
           1       0.53      0.67      0.59       782

    accuracy                           0.89      6370
   macro avg       0.74      0.79      0.76      6370
weighted avg       0.90      0.89      0.89      6370

Resultado sem thredhhold alterado
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      5588
           1       0.57      0.57      0.57       782

    accuracy                           0.89      6370
   macro avg       0.76      0.76      0.76      6370
weighted avg       0.89      0.89      0.89      6370



C:\Users\jvmil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [09:51:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Coluna não usada:  previous
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93      5588
           1       0.53      0.66      0.59       782

    accuracy                           0.89      6370
   macro avg       0.74      0.79      0.76      6370
weighted avg       0.90      0.89      0.89      6370

Resultado sem thredhhold alterado
Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      5588
           1       0.57      0.54      0.55       782

    accuracy                           0.89      6370
   macro avg       0.75      0.74      0.75      6370
weighted avg       0.89      0.89      0.89      6370

{'base_model': 0.5899942495687176, 'age': 0.5876347135564379, 'job': 0.5880361173814899, 'marital': 0.5836139169472503, 'education': 0.5909345271404589, 'default': 0.5912743972445464, 'balance': 0.581235697